In [109]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [110]:
df = pd.read_csv("/Users/nicolasdebie/Master thesis/Benchmarking-GNN-ILP/docker/Benchmark/krk/relational/krk.csv")
print(len(df["class"].value_counts()))
# sample 40 random rows
df = df.sample(200, random_state=42)

# class_weights = [1 if label == "illegal" else 0.3 for label in df["class"]]
# #df_train , df_test = train_test_split(df, test_size=0.1,random_state=42,stratify=df["class"])

# print(df_train["class"].value_counts())
# print(df_test["class"].value_counts())

# df_test["class"] = df_test["class"].map({"illegal": "neg", "normal": "pos"})
# df_test.to_csv("/Users/nicolasdebie/Master thesis/Benchmarking-GNN-ILP/docker/Benchmark/krk/relational/test/krk.csv")

2


In [111]:
### popper
output_folder = os.path.join("docker","Benchmark","krk","original","logic","popper")
output_folder = "/Users/nicolasdebie/Master thesis/Benchmarking-GNN-ILP/docker/Benchmark/krk/original/logic/popper/"

# examples
exs = []
for i in range(0,len(df_train)):
    if df_train["class"].iloc[i] == "legal":
        exs.append(f"pos(krk(k{i})).\n")
    else:
        exs.append(f"neg(krk(k{i})).\n")

with open(os.path.join(output_folder,"exs.pl"),"w") as f:
    for ex in exs:
        f.write(ex)


# bk
bk = []
bk_test = []
bk.append("king(k).\n")
bk_test.append("king(k).\n")
bk.append("rook(r).\n")
bk_test.append("rook(r).\n")
bk.append("white(w).\n")
bk_test.append("white(w).\n")
bk.append("black(b).\n")
bk_test.append("black(b).\n")
bk.append("distance((X1,Y1),(X2,Y2),D) :- D1 is abs(X1-X2), D2 is abs(Y1-Y2),D is max(D1,D2).\n")
bk_test.append("distance((X1,Y1),(X2,Y2),D) :- D1 is abs(X1-X2), D2 is abs(Y1-Y2),D is max(D1,D2).\n")
bk.append("one(1).\n")
bk_test.append("one(1).\n")


for i in range(0,len(df_train)):
    white_king_x = int(df_train["white_king_file"].iloc[i])
    white_king_y = int(df_train["white_king_rank"].iloc[i])
    bk.append(f"cell(k{i},({white_king_x},{white_king_y}),w,k).\n")

    white_rook_x = int(df_train["white_rook_file"].iloc[i])
    white_rook_y = int(df_train["white_rook_rank"].iloc[i])
    bk.append(f"cell(k{i},({white_rook_x},{white_rook_y}),w,r).\n")

    black_king_x = int(df_train["black_king_file"].iloc[i])
    black_king_y = int(df_train["black_king_rank"].iloc[i])
    bk.append(f"cell(k{i},({black_king_x},{black_king_y}),b,k).\n")

for i in range(0,len(df_test)):
    white_king_x = int(df_test["white_king_file"].iloc[i])
    white_king_y = int(df_test["white_king_rank"].iloc[i])
    bk_test.append(f"cell(k{i},({white_king_x},{white_king_y}),w,k).\n")

    white_rook_x = int(df_test["white_rook_file"].iloc[i])
    white_rook_y = int(df_test["white_rook_rank"].iloc[i])
    bk_test.append(f"cell(k{i},({white_rook_x},{white_rook_y}),w,r).\n")

    black_king_x = int(df_test["black_king_file"].iloc[i])
    black_king_y = int(df_test["black_king_rank"].iloc[i])
    bk_test.append(f"cell(k{i},({black_king_x},{black_king_y}),b,k).\n")

with open(os.path.join(output_folder,"bk.pl"),"w") as f:
    print(len(bk))
    for b in bk:
        f.write(b)

# write the test bk to another test file 
with open("/Users/nicolasdebie/Master thesis/Benchmarking-GNN-ILP/docker/Benchmark/krk/original/logic/"+"krk_test.kb","w") as f:
    print(len(bk_test))
    for b in bk_test:
        f.write(b)

# bias 
bias = []
bias.append("max_vars(8).\n")
bias.append("max_body(8).\n")
bias.append("max_clauses(1).\n")

type_bias = [
    "head_pred(krk,1).\n",
    "body_pred(distance,3).\n",
    "body_pred(cell,4).\n",
    "body_pred(king,1).\n",
    "body_pred(rook,1).\n",
    "body_pred(white,1).\n",
    "body_pred(black,1).\n",
    "body_pred(one,1).\n",
    "type(krk,(state,)).\n",
    "type(distance,(pos, pos, integer)).\n",
    "type(cell,(state, pos, color, piecetype)).\n",
    "type(king,(piecetype,)).\n",
    "type(rook,(piecetype,)).\n",
    "type(white,(color,)).\n",
    "type(black,(color,)).\n",
    "type(one,(integer,)).\n",
    "direction(krk,(in,)).\n",
    "direction(distance,(in, in, out)).\n",
    "direction(cell,(in, out, out, out)).\n",
    "direction(king,(in,)).\n",
    "direction(rook,(in,)).\n",
    "direction(white,(in,)).\n",
    "direction(black,(in,)).\n",
    "direction(one,(in,)).\n"
]

bias = bias + type_bias

with open(output_folder+"bias.pl","w") as f:
    for b in bias:
        f.write(b)


276
36


In [112]:
### tilde
output_folder = "/Users/nicolasdebie/Master thesis/Benchmarking-GNN-ILP/docker/Benchmark/krk/original/logic/tilde/"

# examples
kb = []
for i in range(0,len(df_train)):
    if df_train["class"].iloc[i] == "legal":
        kb.append(f"krk({i},pos).\n")
    else:
        kb.append(f"krk({i},neg).\n")

kb.append("king(k).\n")
kb.append("rook(r).\n")
kb.append("white(w).\n")
kb.append("black(b).\n")
kb.append("distance((X1,Y1),(X2,Y2),D) :- D1 is abs(X1-X2), D2 is abs(Y1-Y2),D is max(D1,D2).\n")
kb.append("one(1).\n")

for i in range(0,len(df_train)):
    white_king_x = int(df_train["white_king_file"].iloc[i])
    white_king_y = int(df_train["white_king_rank"].iloc[i])
    kb.append(f"cell({i},({white_king_x},{white_king_y}),w,k).\n")

    white_rook_x = int(df_train["white_rook_file"].iloc[i])
    white_rook_y = int(df_train["white_rook_rank"].iloc[i])
    kb.append(f"cell({i},({white_rook_x},{white_rook_y}),w,r).\n")

    black_king_x = int(df_train["black_king_file"].iloc[i])
    black_king_y = int(df_train["black_king_rank"].iloc[i])
    kb.append(f"cell({i},({black_king_x},{black_king_y}),b,k).\n")

with open(os.path.join(output_folder,"krk.kb"),"w") as f:
    for b in kb:
        f.write(b)


# bias 
tilde_bias = [
    "predict(krk(+id,-class)).\n",
    "warmode(king(+-piece)).\n",
    "warmode(rook(+-piece)).\n",
    "warmode(white(+-color)).\n",
    "warmode(black(+-color)).\n",
    "warmode(distance((+-x,+-y),(+-x,+-y),-integer)).\n",
    "warmode(cell(+id,(+-x,+-y),+-color,+-piecetype)).\n",
    "warmode(one(-integer)).\n",
    "typed_language(yes).\n",
    "type(krk(id,class)).\n",
    "type(king(piece)).\n",
    "type(rook(piece)).\n",
    "type(white(color)).\n",
    "type(black(color)).\n",
    "type(distance((int,int),(int,int),integer)).\n",
    "type(cell(id,(int,int),color,piecetype)).\n",
    "type(one(int)).\n",
]

with open(output_folder+"krk.s","w") as f:
    for b in tilde_bias:
        f.write(b)